In [1]:
import pandas as pd
from pandas import DataFrame as df
import IPython as ip

import seaborn as sns
sns.set(color_codes=True)

import matplotlib.pyplot as plt
# Set input dir
import sys
sys.path.append("/home/mschlupp/pythonTools")
files_dir = "files/"

In [8]:
ga_full = pd.read_csv(files_dir+"traintest_fullevt.csv")

In [10]:
ga_hasEvts = ga_full[ga_full.hasEvents>0]

In [9]:
ga_noEvts = ga_full[ga_full.hasEvents==0]

In [12]:
ga_noEvts.drop(["hasEvents", "nEvts", "longitude_mean"
                , "longitude_variance", "latitude_mean"
                , "latitude_variance", "usageTime_mean"
                , "usageTime_variance", "usageDay_mean"
                ,"usageDay_variance"], axis=1).head(2)

,age,device_id,gender,group,isTrain,phone_brand,device_model
0,35,-8076087639492063270,M,M32-38,1,小米,MI 2
1,35,-2897161552818060146,M,M32-38,1,小米,MI 2


In [7]:
ga_hasEvts.head(2)

,age,device_id,gender,group,isTrain,phone_brand,device_model,hasEvents,nEvts,longitude_mean,longitude_variance,latitude_mean,latitude_variance,usageTime_mean,usageTime_variance,usageDay_mean,usageDay_variance
2,35,-8260683887967679142,M,M32-38,1,小米,MI 2,1,1,0.000000,-1.000000,0.000000,-1.000000,14.000000,-1.000000,3.000000,-1.000000
11,37,7477216237379271436,F,F33-42,1,华为,荣耀6 plus,1,7,102.491429,2042.540348,27.214286,144.008929,14.571429,2.285714,5.142857,5.142857
